# Import Dependencies

In [1]:
!pip install -q findspark

In [2]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 56 kB/s 
     |████████████████████████████████| 199 kB 72.2 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=d2fa5d41cef3c47edf3cc99e3a0c469646da52c4fca7980b21403ed4f551ec09
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [3]:
import findspark
findspark.init()

In [4]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions

In [5]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('project-de') \
    .getOrCreate()

In [6]:
spark

# Extracting Data

In [ ]:
#url = 'https://storage.googleapis.com/wfwijaya-fellowship/bank-marketing.csv'
#df_download = wget.download(url)

In [7]:
!wget 'https://storage.googleapis.com/wfwijaya-fellowship/bank-marketing.csv'

--2022-10-14 02:59:04--  https://storage.googleapis.com/wfwijaya-fellowship/bank-marketing.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.98.128, 74.125.197.128, 74.125.142.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.98.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4928746 (4.7M) [text/csv]
Saving to: ‘bank-marketing.csv’

bank-marketing.csv  100%[===================>]   4.70M  --.-KB/s    in 0.03s   

2022-10-14 02:59:04 (142 MB/s) - ‘bank-marketing.csv’ saved [4928746/4928746]



In [8]:
df = spark.read.csv('bank-marketing.csv', header = True, inferSchema = True)
df1 = spark.read.csv('bank-marketing.csv', header = True, inferSchema = True).createOrReplaceTempView('bank_marketing')

In [9]:
df.show(5)

+---+---------+-------+-----------+-------+-------+----+---------+-----+-----------+--------+--------+-----+--------+-----------+------------+--------------+-------------+---------+-----------+---+
|age|      job|marital|  education|default|housing|loan|  contact|month|day_of_week|duration|campaign|pdays|previous|   poutcome|emp.var.rate|cons.price.idx|cons.conf.idx|euribor3m|nr.employed|  y|
+---+---------+-------+-----------+-------+-------+----+---------+-----+-----------+--------+--------+-----+--------+-----------+------------+--------------+-------------+---------+-----------+---+
| 56|housemaid|married|   basic.4y|     no|     no|  no|telephone|  may|        mon|     261|       1|  999|       0|nonexistent|         1.1|        93.994|        -36.4|    4.857|     5191.0| no|
| 57| services|married|high.school|unknown|     no|  no|telephone|  may|        mon|     149|       1|  999|       0|nonexistent|         1.1|        93.994|        -36.4|    4.857|     5191.0| no|
| 37| serv

In [10]:
df.dtypes

[('age', 'int'),
 ('job', 'string'),
 ('marital', 'string'),
 ('education', 'string'),
 ('default', 'string'),
 ('housing', 'string'),
 ('loan', 'string'),
 ('contact', 'string'),
 ('month', 'string'),
 ('day_of_week', 'string'),
 ('duration', 'int'),
 ('campaign', 'int'),
 ('pdays', 'int'),
 ('previous', 'int'),
 ('poutcome', 'string'),
 ('emp.var.rate', 'double'),
 ('cons.price.idx', 'double'),
 ('cons.conf.idx', 'double'),
 ('euribor3m', 'double'),
 ('nr.employed', 'double'),
 ('y', 'string')]

In [11]:
df_clean = df.select('age', 'job', 'marital', 'education', 'default', 'housing', 'loan').na.drop()

In [12]:
df_clean.show()

+---+-----------+--------+-------------------+-------+-------+----+
|age|        job| marital|          education|default|housing|loan|
+---+-----------+--------+-------------------+-------+-------+----+
| 56|  housemaid| married|           basic.4y|     no|     no|  no|
| 57|   services| married|        high.school|unknown|     no|  no|
| 37|   services| married|        high.school|     no|    yes|  no|
| 40|     admin.| married|           basic.6y|     no|     no|  no|
| 56|   services| married|        high.school|     no|     no| yes|
| 45|   services| married|           basic.9y|unknown|     no|  no|
| 59|     admin.| married|professional.course|     no|     no|  no|
| 41|blue-collar| married|            unknown|unknown|     no|  no|
| 24| technician|  single|professional.course|     no|    yes|  no|
| 25|   services|  single|        high.school|     no|    yes|  no|
| 41|blue-collar| married|            unknown|unknown|     no|  no|
| 25|   services|  single|        high.school|  

In [13]:
df_cluster = spark.sql("SELECT age, job, marital, education, default, housing, loan FROM bank_marketing CLUSTER BY age")
df_cluster.show()

+---+-------+-------+-----------+-------+-------+-------+
|age|    job|marital|  education|default|housing|   loan|
+---+-------+-------+-----------+-------+-------+-------+
| 17|student| single|    unknown|     no|    yes|     no|
| 17|student| single|   basic.9y|     no|    yes|     no|
| 17|student| single|   basic.9y|     no|    yes|     no|
| 17|student| single|   basic.9y|     no|unknown|unknown|
| 17|student| single|    unknown|     no|     no|    yes|
| 18|student| single|high.school|     no|     no|     no|
| 18|student| single|high.school|     no|    yes|    yes|
| 18|student| single|   basic.4y|     no|     no|     no|
| 18|student| single|   basic.4y|     no|    yes|    yes|
| 18|student| single|high.school|     no|     no|     no|
| 18|student| single|high.school|     no|    yes|     no|
| 18|student| single|   basic.9y|     no|    yes|     no|
| 18|student| single|   basic.6y|     no|    yes|     no|
| 18|student| single|    unknown|     no|     no|     no|
| 18|student| 

# Write to csv

In [14]:
df_cluster.coalesce(1).write.csv("bank_marketing_clean", header=True)

In [15]:
df_clean.coalesce(1).write.csv("bank_marketing_clean_data", header = True)